In [ ]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.experimental.query_engine.pandas.pandas_query_engine import PandasQueryEngine
import pandas as pd
import textwrap

In [ ]:


def formatar_texto(texto):
  texto_formatado = textwrap.fill(texto, width=100)
  return texto_formatado

In [ ]:
df = pd.read_csv("vendas.csv")

In [ ]:
df.head()

In [ ]:

ollama_llm = Ollama(
    model="llama3.2:3b", 
    request_timeout=120.0, 
    context_window=10240, 
    temperature=0.05, # It scaling the logits before they are turned into probabilities (passed to the softmax function)
    max_tokens=10240,
    top_k=5, # It force the mode select the top k tokens with the highest probability
    # top_p=0.99, # It instruct the model to select from the top p percent of tokens
    # frequency_penalty=0.5, # It penalizes the model for using the same token multiple times in a single response
    # presence_penalty=0.5, # It penalizes the model for using the same token multip
    #repeat_penalty=0.5, # It penalizes the model for using the same token multiple times in a single response
)

Settings.llm = ollama_llm


In [ ]:
query_engine = PandasQueryEngine(df=df, synthesize_response=True, verbose=True)

In [ ]:
perguntas = [
    {
        "pergunta": "Qual é a forma de pagamento mais utilizada pelos clientes?", 
        "resposta_esperada": "Carteira Digital"
    },
    {
        "pergunta": "Qual é o tipo de produto com maior quantidade por filial?", # The model answer better when use the words: em cada filial
        "resposta_esperada": "(A, Casa), (B, Esportes e Viagem), (C, Alimentos e Bebidas)"
    },
    {
        "pergunta": "Qual é a forma de pagamento mais utilizada pelos clientes em cada filial?", 
        "resposta_esperada": "(A, Carteira Digital), (B, Carteira Digital), (C, Dinheiro)"
    },
    {
        "pergunta": "Em quais cidades temos filiais?", 
        "resposta_esperada": "['Santo André' 'São Caetano' 'São Bernardo do Campo']"
    },
    {
        "pergunta": "Qual é o preço unitário médio de cada tipo de produto?",
        "resposta_esperada": """
            (Alimentos e Bebidas, 56.008851), 
            (Casa, 55.316937), 
            (Eletrônicos, 53.551588), 
            (Esportes e Viagem, 56.993253),
            (Moda, 57.153652),
            (Saúde e Beleza 54.854474)
        """
    },
    {
        "pergunta": "Qual é a média de valor de compra por tipo de cliente?",
        "resposta_esperada": """
            (Membro 327.791305),
            (Normal 318.122856)

        """
    },
    {
        "pergunta": "Qual é a filial com maior faturamento?",
        "resposta_esperada": "C"
    },
    {
        "pergunta": "Qual é a avaliação média por filial?",
        "resposta_esperada": """"
            (A, 7.027059),
            (B, 6.818072),
            (C, 7.072866)
        """
    },
    {
        "pergunta": "Quais filiais têm uma avaliação média acima de 7?",
        "resposta_esperada": """
            df.groupby("filial")["avaliacao"].mean().reset_index().query("avaliacao > 7")
            (A, B)
        """
    },
    {
        "pergunta": "Crie um gráfico do tipo boxplot das avaliações por filial para comparar a distribuição desses dados",
        "resposta_esperada": """
            df.boxplot(column="avaliacao", by="filial")
        """
    },
    
]

In [ ]:
for pergunta in perguntas:
    print(f'Pergunta: {pergunta["pergunta"]}')
    print(f'Resposta Esperada: {formatar_texto(pergunta["resposta_esperada"])}')
    resposta = query_engine.query(pergunta["pergunta"]).response
    print(f'Resposta: {formatar_texto(resposta)}')
    print('---')
    print('---')
    print('---')

In [ ]:
# response = query_engine.query('Você pode plotar a distribuição das avaliações?')

In [ ]:
# df.avaliacao.plot(kind='hist')